In [2]:
import os, pdfplumber, pprint, openai
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from itertools import chain
from dotenv import load_dotenv

In [3]:
load_dotenv('../.env')
openai.api_key = os.getenv("CHATGPT-RECEIPT")

#### Assistant 생성

In [4]:
client = OpenAI(api_key=os.getenv("CHATGPT-RECEIPT"))

## vector store 생성
file = client.files.create(
    file = open('../pdf/r-d-innovation.pdf', 'rb'),
    purpose = 'assistants'
)
vector_store = client.beta.vector_stores.create(
    name="R&D Innovation"
)
# 파일을 Vector Store에 추가
client.beta.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)

In [9]:
# Assistant 생성
assistant = client.beta.assistants.create(
    name = "My Assistant",
    instructions = """
                넌 국가연구개발 혁신법에 대해 잘 아는 행정 관련 전문가야.
                만약 Q&A에 해당 내용이 있으면 이에 대해 자세하고 친절하게 답변해줘.
                정확하다고 생각되는 답을 찾았으면, 가장 먼저 "예" 혹은 "아니오"로 답변을 시작하고, 그렇지 않으면 "모르겠어"라고 말해줘.
                답한 내용의 근거를 정확하게 말해주되 문서내 검색한 내용만 답해 줘.
                마침표 다음에 개행 문자를 넣어줘.
    """,
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"}
    ],
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    }
)

In [10]:
# Assistant ID 출력 및 저장
print(f'assistant id = {assistant.id}')
assistant_id = assistant.id
# Thread 생성 및 저장
thread = client.beta.threads.create()
print(f'thread id = {thread.id}')
thread_id = thread.id

# thread_id = 'thread_Twa2ruyrfpBGhOqESAnAYn6W'
# assistant_id = 'asst_7J3TrQfqCaD5dWXZpu7665l5'

thread id = thread_Twa2ruyrfpBGhOqESAnAYn6W
assistant id = asst_7J3TrQfqCaD5dWXZpu7665l5


#### Assistant에 질의하기

In [8]:
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="공동연구기관 변경을 하기 위해서는 어떤 절차를 밟아야 하나요?"
)
# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id
)
# Run 완료 대기
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
# 결과 메시지 가져오기
messages = client.beta.threads.messages.list(thread_id=thread_id)
# 결과 출력
for message in messages:
    if message.role == "assistant":
        print(message.content[0].text.value)

공동연구개발기관 변경 절차에 관한 내용은 다음과 같습니다:

1. **상호협의(승인) 필요**: 공동연구개발기관 변경은 중요한 사항으로, 변경하고자 할 때는 상호협의 후 승인을 받아야 합니다. 이 과정은 실무상 승인 절차와 동일하게 진행됩니다.

2. **협약 변경 신청**: 중요한 협약 변경 사항에는 부처 또는 전문기관으로부터 협약변경 심사 및 승인을 받는 절차가 포함됩니다. 국가연구개발사업 업무 대행 협약에 따라 전문기관이 협약 업무를 대행하는 경우가 있으며, 이 때는 전문기관에서 협약변경 심사 및 승인을 수행할 수 있습니다.

3. **정보 변경 수준에서의 변경**: 선정 후 협약 전이나 협약 후 특정 상황에서 지속적 수행이 어렵게 될 경우, 소관 중앙행정기관(전문기관)의 장과 협의하여 정보 변경 수준에서 변경할 수도 있습니다【4:2†source】.

공동연구개발기관 변경 시 해당 절차를 따라 정확하게 신청 및 승인을 받는 과정이 필요함을 알 수 있습니다.
공동연구개발기관 변경 절차에 관한 상세한 답변은 다음과 같습니다.

공동연구개발기관을 변경하려는 경우, 혁신법 제11조제2항에 따라 협약 변경이 가능하며, 이는 중요한 사항의 협약 변경에 해당합니다. 변경 절차를 준수하기 위해서는 다음과 같은 절차를 밟아야 합니다:

1. **상호협의(승인) 필요**: 공동연구개발기관의 변경은 상호협의 후 승인을 받아야 하는 중요한 사항입니다. 이 과정은 사실상 상호 협의 사항이라 할 수 있으며, 실무상 승인 절차와 동일하게 진행됩니다【4:2†source】.

2. **협약 변경 신청**: 중요한 협약 변경 사항에 관해서는 협약 당사자가 부처 또는 전문기관으로부터 협약변경 심사 및 승인을 받아 협약 변경을 처리해야 합니다. 국가연구개발사업 업무 대행 협약에 따라 전문기관이 협약의 업무를 대행하는 경우, 전문기관에서 협약변경 심사 및 승인을 수행할 수 있습니다【4:2†source】.

3. **정보 변경 수준의 변경 가능성**: 선정 후 협약 전이나, 특정 상황에서 지속

In [9]:
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="""
        {
            "상호명": "고향옥",
            "날짜": "2024-12-13",
            "항목": [
                {"이름": "세트 A", "가격": 13000},
                {"이름": "세트 B", "가격": 12000}
            ],
            "총액": 25000
        }
        이런 영수증 내역이 있을 때, 연구비 어떤 항목으로 처리해야 하나요?
        상호를 검색해서 업종을 판단해 주세요.
        단답형으로 항목만 답변해 주세요. (ex, 출장비)
        정확한 항목이 없을 경우, "판단 필요"라고 답변해 주세요.
        판단이 필요한 경우에는 이유를 말해주세요.
    """
)

# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id
)

# Run 완료 대기
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)

# 결과 메시지 가져오기
messages = client.beta.threads.messages.list(thread_id=thread_id)
# 결과 출력
for message in messages:
    if message.role == "assistant":
        print(message.content[0].text.value)

판단 필요.

이유: 영수증 내역에 기반한 연구비 처리 항목 결정을 위해서는 '고향옥'이라는 상호의 업종 정보가 필요합니다. 제공된 정보만으로는 '고향옥'이 어떤 종류의 서비스나 상품을 제공하는지 명확하지 않습니다. 예를 들어, '고향옥'이 음식점일 경우 '회의비' 또는 '업무추진비'로 처리할 수도 있고, 다른 업종일 경우 이에 맞는 적절한 연구비 항목이 필요할 것입니다. 따라서 상호를 통한 업종 검색 후 정확한 연구비 항목을 결정하는 것이 필요합니다.
공동연구개발기관 변경 절차에 관한 내용은 다음과 같습니다:

1. **상호협의(승인) 필요**: 공동연구개발기관 변경은 중요한 사항으로, 변경하고자 할 때는 상호협의 후 승인을 받아야 합니다. 이 과정은 실무상 승인 절차와 동일하게 진행됩니다.

2. **협약 변경 신청**: 중요한 협약 변경 사항에는 부처 또는 전문기관으로부터 협약변경 심사 및 승인을 받는 절차가 포함됩니다. 국가연구개발사업 업무 대행 협약에 따라 전문기관이 협약 업무를 대행하는 경우가 있으며, 이 때는 전문기관에서 협약변경 심사 및 승인을 수행할 수 있습니다.

3. **정보 변경 수준에서의 변경**: 선정 후 협약 전이나 협약 후 특정 상황에서 지속적 수행이 어렵게 될 경우, 소관 중앙행정기관(전문기관)의 장과 협의하여 정보 변경 수준에서 변경할 수도 있습니다【4:2†source】.

공동연구개발기관 변경 시 해당 절차를 따라 정확하게 신청 및 승인을 받는 과정이 필요함을 알 수 있습니다.
공동연구개발기관 변경 절차에 관한 상세한 답변은 다음과 같습니다.

공동연구개발기관을 변경하려는 경우, 혁신법 제11조제2항에 따라 협약 변경이 가능하며, 이는 중요한 사항의 협약 변경에 해당합니다. 변경 절차를 준수하기 위해서는 다음과 같은 절차를 밟아야 합니다:

1. **상호협의(승인) 필요**: 공동연구개발기관의 변경은 상호협의 후 승인을 받아야 하는 중요한 사항입니다. 이 과정은 사실상 상호 협의 사항이라 할 수 있으며, 실무상 승인 절차와 동